# Snippets SDK


We store our subscriptions inside a dictionary: `subscriptions_dict`.

The APIs return a lot of crap. The JSON responses are saved as dictionaries. From there, I tried to access some 'useful' information.

In [298]:
# Initializer 
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json
import os, requests
import numpy as np

# Initializer
# Subscriptions
subscriptions_dict = {'computer_vision':'c0fd5de0a7eb4ad9a241d1fe1f4cd989',
                      'face_api': 'cd91c79a18534dc58d7eae4608ea3d3d',
                      'speech_api': 'd3015735e75546cd9af885976b8e4c54',
                      'text_analytics': 'bcf64cde5aad4e09856471aa6530026f',
                      'luis':'2a2b546798854097ace2140bf961a493'}

# Request Headers
headers = {'Content-Type': 'applications/json',
            'Ocp-Apim-Subscription-Key': subscriptions_dict['computer_vision']}

# Content body to be sent, many forms, to be redefined
body = "{'url':'https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg'}"
# _ _ _ _  _ _ 

### Modifying requests

Two global variables to be used for all the requests:

- `body` : many forms, depends on the API
- `headers`: dict with two keys: 
    - the content type: 'application/json' or 'application/octet-stream'.
    - the API key of your resource.


In [129]:
# The target body for is changed using these two functions
def change_headers(contentType='application/json', api='computer_vision'):
    '''At the beginning of every function to ensure proper settings'''
    global headers
    headers = {'Content-Type': contentType,
               'Ocp-Apim-Subscription-Key':subscriptions_dict[api]}

# Camera for Pi
# Takes picture by calling the raspistill command in the terminal and defines it as body
def body_take_picture(name='me.png'):
    """defines body as binary image data of the picture taken, 
       required for comptuer vision api"""
    global body, headers
    headers['Content-Type'] = 'application/octet-stream'
    os.system('raspistill -o {}'.format("/home/pi/Robuzure/{}.png".format(name)))
    body = open('/home/pi/Robuzure/{}.png'.format(name), 'rb')

# For Test in PC
def body_url_image(url='https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg'):
    global headers, body
    headers['Content-Type'] = 'application/json'
    body = "{'url':'%s'}"%url
    return body
    

In [294]:
# This defines body as a dictionary for application/json requests for CV: URL based
body_url_image()

"{'url':'https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg'}"

In [295]:
body

"{'url':'https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg'}"

To be able to use different APIs in the same environment without errors, I made the `change_headers` function to specify what `contentType` and `api` to use. The `api` argument access your key from the resource `subscriptions_dict[api]`.

When changing from one API to another, use `change_headers` to configure the request correctly.

For example:
```python
# Configure the head of the request for ComputerVision and URL based images
change_headers(contentType='application/json',api='computer_vision')
# Define global `body` as a dictionary {'url':'https...'} 
body_url_image(url='https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg')
# call a functions from the CV API, using `body`
describe_image(body, number_of_descriptions=3)

```

**In the Pi**, the function `body_take_picture` tells the pi to take a picture and  define it as the global `body` (as binary image data). The global body can be then sent to the APIs:

```python
# Configure the head of the request for Computer Vision and binary image data
change_headers(contentType='application/octet-stream', api='computer_vision')
# Define global body as binary image data from the image me.png
body_take_picture(name='me.png') 
# call function from the CV API, using body
describe_image(body, number_of_descriptions=3)
```


> I really hope this is not too convoluted. Check the examples below!

## Computer Vision and Face API

In [132]:
def describe_image(img, number_of_descriptions=1):
    '''Returns a number_of_descriptions from an image'''
    params = urllib.parse.urlencode({
    # Request parameters
    'maxCandidates': "{}".format(number_of_descriptions),
    'language': 'en'})

    try:
        conn = http.client.HTTPSConnection('westeurope.api.cognitive.microsoft.com')
        # img recieves the binary image (body)
        conn.request("POST", "/vision/v2.0/describe?%s" % params, img, headers)
        response = conn.getresponse()
        data = response.read()
        r = json.loads(data.decode())
        
        for i in range(len(r['description']['captions'])):
            print(r['description']['captions'][i]['text'])

        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [312]:
# redefines body
body_url_image(url='https://pbs.twimg.com/profile_images/1062098998746644480/JTIpfWME_400x400.jpg')
# set up headers
change_headers(contentType='application/json', api='computer_vision')
# function using body
describe_image(body, number_of_descriptions=4)

a person standing in front of a large rock
a person standing on a rock
a person standing on a rock
a person standing in front of a rock


In [138]:
def detect_objects(img):
    ''''''
    global found_objects
    found_objects = []
    params = urllib.parse.urlencode({})
    try:
        conn = http.client.HTTPSConnection('westeurope.api.cognitive.microsoft.com')
        conn.request("POST", "/vision/v2.0/detect?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read()
        r = json.loads(data.decode())
        #print(data)
        print(r.keys())
        print("In the image of size {} by {} pixels, {} objects were detected".format(r['metadata']['width'], r['metadata']['height'], 
                                                                                      len(r['objects']))) 
        for i in r['objects']:
            # acessing the rectangle key
            found_objects.append(i['object'])
            print('{} detected at region {}'.format(i['object'], i['rectangle']))
        # print(found_objects) can be read by speaker
        # DRAW BOUNDING BOXES into Image with CV2?
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

change_headers(api='computer_vision')

In [313]:
# To analyze the same image in the same api, no need to redefine body or headers
detect_objects(body)

dict_keys(['objects', 'requestId', 'metadata'])
In the image of size 400 by 400 pixels, 3 objects were detected
tree detected at region {'x': 75, 'y': 2, 'w': 68, 'h': 93}
headwear detected at region {'x': 130, 'y': 71, 'w': 117, 'h': 68}
person detected at region {'x': 93, 'y': 70, 'w': 284, 'h': 329}


In [140]:
def analyze_image(img, visualFeatures='Description, Categories, Faces'):
    '''visualFeatures: a string of comma separated keywords 'Brands, Adult, Objects'
       Category of Image
       Tags of the image'''
    global speak
    change_headers(api='computer_vision')
    params = urllib.parse.urlencode({
        # Request parameters
        'visualFeatures': '{}'.format(visualFeatures),
        'details': '',
        'language': 'en'})

    try:
        conn = http.client.HTTPSConnection('westeurope.api.cognitive.microsoft.com')
        conn.request("POST", "/vision/v2.0/analyze?%s" % params, img, headers)
        response = conn.getresponse()
        data = response.read()
        r = json.loads(data.decode())

        # Handling Response in some useful manner. JSON has a lot of keys.
        if r['faces'] == []:
            print('No faces encountered')
        else:
            print('At least a face was detected')
        print("Category of image: {}".format(r['categories'][0]['name']))
        
        # extracting tags
        print("Tags found in the image {}".format(r['description']['tags']))
        # send to some text generator service

        if "Description" in visualFeatures:
            speak = (r['description']['captions'][0]['text'])

        print(speak)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [306]:
# no need to redefine headers or body
analyze_image(body)

At least a face was detected
Category of image: people_
Tags found in the image ['outdoor', 'person', 'rock', 'young', 'woman', 'holding', 'man', 'standing', 'hat', 'smiling', 'wearing', 'boy', 'riding', 'snow', 'walking', 'large', 'rocky', 'park', 'dirt', 'board', 'playing', 'skiing']
a person standing in front of a large rock


## Face API

In [316]:
def detect_face(img):
    global info
    
    """Find attributes of a human face age, gender, emotions etc."""
    params = urllib.parse.urlencode({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'false',
        'returnFaceAttributes': 'age,gender,smile,facialHair,glasses,emotion,hair',
        'recognitionModel': 'recognition_01',
        'returnRecognitionModel': 'false',
    })

    try:
        conn = http.client.HTTPSConnection('westeurope.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/detect?%s" % params, img, headers)
        response = conn.getresponse()
        data = response.read()
        conn.close()
        #print(data)
        r = json.loads(data.decode())
    
        print('{} face(s) were found'.format(len(r)))

        for face_info in r:
            attributes = face_info['faceAttributes']
            #print("Face Info ", attributes, '\n')
            highest_emotion_index = np.argmax(list(attributes['emotion'].values()))
            #print(highest_emotion_index)
            info = "A {} of around {} years with a {} of {} percent".format(attributes['gender'], 
                                                                            attributes['age'], 
                                                                            list(attributes['emotion'].keys())[highest_emotion_index],
                                                                           int(max(attributes['emotion'].values())*100))
            print(info)
            
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [317]:
# Now we need to change the api from computer_vision to face_api so we redefine headers
# the body is the same, as we want to analyze the same image
change_headers(contentType='application/json',api='face_api')
detect_face(body)

1 face(s) were found
A male of around 33.0 years with a happiness of 100 percent


## Text Analytics

The function `text_body` redefines `body` in the correct format to send to the Text Analytics API. I put inside the functions for convenience. The `body` variable is redefined with the `text` argument of the functions.

In [318]:
TextToAnalyze = 'Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.\nNow we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battlefield of that war.\nWe have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.\nBut, in a larger sense, we can not dedicate, we can not consecrate, we can not hallow this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract.\nThe world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced.\nIt is rather for us to be here dedicated to the great task remaining before us that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion, that we here highly resolve that these dead shall not have died in vain; that this nation, under God, shall have a new birth of freedom and that government of the people, by the people, for the people, shall not perish from the earth.'

def text_body(TextToAnalyze=TextToAnalyze):
    """To create a body of a single text to be analyzed. The (same Id to track more documents)"""
    global body
    body = {
        # to this list you can add more dictionaries {lang, id, text..} to analyze more than one text
      "documents": [
        {
            "id": "TextTest",
            "text": "'{}'".format(TextToAnalyze)
        }]}
    
text_body()

In [176]:
def detect_language(text):
    '''Only for one '''
    textAnalyticsURI = 'westeurope.api.cognitive.microsoft.com'
    change_headers(api='text_analytics')
    params = urllib.parse.urlencode({})
    text_body(text)
    try:
        conn = http.client.HTTPSConnection(textAnalyticsURI)
        conn.request("POST", "/text/analytics/v2.0/languages?%s" % params, str(body), headers)
        response = conn.getresponse()
        data = response.read()
        r = json.loads(data.decode())
        print(r['documents'][0]['detectedLanguages'][0]['name'])
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [336]:
change_headers(contentType='application/json',api='text_analytics')
# Redefines body and calls function
detect_language(text='bon dia com et dius?')

Catalan


In [181]:
def key_phrases(text):
    # Define the parameters
    textAnalyticsURI = 'westeurope.api.cognitive.microsoft.com'
    change_headers(api='text_analytics')
    params = urllib.parse.urlencode({})
    text_body(text)
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(textAnalyticsURI)
        conn.request("POST", "/text/analytics/v2.0/keyPhrases?%s" % params, str(body), headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")

        # 'data' contains the JSON response, which includes a collection of documents.
        parsed = json.loads(data)
        for document in parsed['documents']:
            print("Document " + document["id"] + " key phrases:")
            for phrase in document['keyPhrases']:
                print("  " + phrase)
            print("---------------------------")
        conn.close()

    except Exception as e:
        print('Error:')
        print(e)

In [338]:
# redefines body, same headers as before
key_phrases('Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.Now we are engaged in a great civil war')

Document TextTest key phrases:
  fathers
  proposition
  continent
  Liberty
  new nation
  great civil war
  score
  years
  men
---------------------------


In [198]:
def check_sentiment(text):
     # Define the parameters
    textAnalyticsURI = 'westeurope.api.cognitive.microsoft.com'
    change_headers(api='text_analytics')
    params = urllib.parse.urlencode({})
    text_body(text)
    try:
        conn = http.client.HTTPSConnection(textAnalyticsURI)
        conn.request("POST", "/text/analytics/v2.0/sentiment?%s" % params, str(body), headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        parsed = json.loads(data) 
        # Get the numeric score for each document
        for document in parsed['documents']:
            sentiment = "negative"
            # if it's more than 0.5, consider the sentiment to be positive.
            if document["score"] >= 0.5:
                sentiment = "positive"
            print("Document:" + document["id"] + " = " + sentiment)
        conn.close()
    
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [339]:
# redefines body
check_sentiment('Actually pretty normal')

Document:TextTest = positive


In [323]:
check_sentiment('Good but not so amazing')

Document:TextTest = negative


In [326]:
check_sentiment('Good. Bad. Bad. Good. Bad')

Document:TextTest = negative


In [327]:
check_sentiment('Interesting')

Document:TextTest = positive


In [159]:
def find_entities(body):
    params = urllib.parse.urlencode({
    })
    textAnalyticsURI = 'westeurope.api.cognitive.microsoft.com'
    change_headers(api='text_analytics')
    try:
        conn = http.client.HTTPSConnection('westeurope.api.cognitive.microsoft.com')
        conn.request("POST", "/text/analytics/v2.0/entities?%s" % params, str(body), headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [160]:
find_entities(body)

b'{"documents":[{"id":"TextTest","entities":[{"name":"Conceived in Liberty","matches":[{"text":"conceived in Liberty","offset":91,"length":20}],"wikipediaLanguage":"en","wikipediaId":"Conceived in Liberty","wikipediaUrl":"https://en.wikipedia.org/wiki/Conceived_in_Liberty","bingId":"9540782b-ef0a-caed-5657-9706c6b8ce02"},{"name":"Earth","matches":[{"text":"earth","offset":1450,"length":5}],"wikipediaLanguage":"en","wikipediaId":"Earth","wikipediaUrl":"https://en.wikipedia.org/wiki/Earth","bingId":"6ddb3372-4801-5567-321e-e8a53bd774a4"},{"name":"God","matches":[{"text":"God","offset":1319,"length":3}],"wikipediaLanguage":"en","wikipediaId":"God","wikipediaUrl":"https://en.wikipedia.org/wiki/God","bingId":"d38371fb-8068-3139-6cb8-e9cfe3118da0"}]}],"errors":[]}'


## Speech API

The Speech API is a bit different. It requires you to **have an access token**, each one is valid for 10 minutes [ ! ].

You can request an access token like this:

In [333]:
# Use the API key to request an access token
response = requests.post('https://westeurope.api.cognitive.microsoft.com/sts/v1.0/issueToken', headers=headers)
accesstoken = str(response.text)

The `speech_to_text` function receives the path of a wav file and outputs the text.

In [334]:
def speech_to_text(path_to_wav):
    '''Input a wav file, returns text'''
    change_headers(api='speech_api')
    with open("{}".format(path_to_wav), mode="rb") as audio_file:
        audio_data =  audio_file.read()
    
    # Now that we have a token, we can set up the request
    speechToTextEndPoint = "https://westeurope.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1"
    headers = {"Content-type": "audio/wav; codec=audio/pcm; samplerate=16000", 
            "Authorization": "Bearer " + accesstoken}
    params = {"language":"en-US"}
    body = audio_data

    # Connect to server, post the request, and get the result
    response = requests.post(speechToTextEndPoint,data=body, params=params, headers=headers)
    result = str(response.text)
    print(json.loads(result)['DisplayText'])


In [335]:
change_headers(contentType='application/json', api='speech_api')
speech_to_text('/Users/jdchipox/Downloads/DAT263x-demos/Text and Speech/RainSpain.wav')

The rain in Spain stays mainly in the plain.


## [LUIS for Language Understanding](https://azure.microsoft.com/en-us/services/cognitive-services/language-understanding-intelligent-service/)

- [Really cool stuff](https://www.luis.ai/welcome)

- Get intent! We can use it to take pictures, record audio!

I created a very simple model with two intents: 
    - Take Picture
    - Record Audio
    
Then I gave a couple of phrases that specify one intent or the other. Such as:
- `what do you see?` => Take a picture
- `listen to me` => Record Audio


In [287]:
def get_intent(text):
    import requests
    params ={
        # Query parameter
        'q': ''.format(text),
        # Optional request parameters, set to default values
        'timezoneOffset': '0',
        'verbose': 'false',
        'spellCheck': 'false',
        'staging': 'false',
        'subscription-key':'1b91b846370b41e8b9277ab927fb558c'}

    try:

        #r = requests.get('https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/df46b650-b9f9-4489-b9f8-74532e9fd247',headers=headers, params=params)
        r = requests.get('https://westus.api.cognitive.microsoft.com/luis/v2.0/apps/df46b650-b9f9-4489-b9f8-74532e9fd247?verbose=true&timezoneOffset=-360&subscription-key=1b91b846370b41e8b9277ab927fb558c&q={}'.format(text))
        r = json.loads((r.content).decode())
        print(r['topScoringIntent'])

    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [291]:
change_headers(contentType='application/json', api='luis')
get_intent('Take a picture with the camera')

{'intent': 'Take Picture', 'score': 0.9801855}


In [292]:
get_intent('Record my voice')

{'intent': 'Record Audio', 'score': 0.931635}


## TO DO:
- Create a function that records audio in the Raspberry Pi.
- Chain functions together for useful purposes. For example, after recording a wav file as 'audio.wav' with the pi we can use `speech_to_text` to extract the text and then pass this to `get_intent` to get the intention we want, for example Take a Picture.



# Snippets SDK

## Subscriptions

We store our Azure Subscriptions inside a dictionary.

```

```

The APIs return a lot of crap. The JSON responses are saved as dicionaries. I tried to access some 'useful' information.

## Computer Vision API
### Input

The Pi takes a picture with:
```python
 os.system('raspistill -o {}'.format("/home/pi/SavedImg.png"))
```

The image is given as binary data inside the body:

```python
body = open('/home/pi/SavedImg.png', 'rb')
```

### Functions
```python
describe_image(body, number_of_descriptions=3)`
```
```
a crowded city street filled with traffic at night.
a group of people on a city street filled with traffic at night.
a close up of a busy city street filled with traffic at night.
```
___
```python
detect_objects(body)
```
```
In the image of size 1826 by 2436 pixels, 5 objects were detected> taxi detected at region {'x': 1277, 'y': 2057, 'w': 363, 'h': 231}
person detected at region {'x': 0, 'y': 2222, 'w': 174, 'h': 213}
person detected at region {'x': 199, 'y': 2224, 'w': 299, 'h': 206}
person detected at region {'x': 434, 'y': 2176, 'w': 236, 'h': 251}
person detected at region {'x': 826, 'y': 2161, 'w': 255, 'h': 270}

# Would be cool to draw the boxes in the images
```
___
```python
analyze_image(body)
```
```
No faces encountered
Category of image: outdoor_street
Tags found in the image: ['outdoor', 'building', 'street', 'city', 'busy', 'traffic', 'filled', 'people', 'car', 'walking', 'many', 'crowded', 'bunch', 'group', 'night', 'large', 'table', 'light', 'man', 'standing', 'umbrella', 'riding', 'tall', 'sign', 'rain', 'road']

a crowded city street filled with traffic at night
```
## Face API
Receives the same type of `body` as before.

```python
face_detect(body)
```

```

```


## Speech 
```python
speech_to_text('</Users/jdchipox/Downloads/DAT263x-demos/Text and Speech/RainSpain.wav>')
```
```
The rain in Spain stays mainly in the plain.
```

## Text Analytics

We need to give the text inside a dictionary as follows:

### Input
```python
TextToAnalyze = 'Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty...'

body = {
  "documents": [
    {
        "id": "TextTest",
        "text": "'{}'".format(TextToAnalyze)
    }]}
```
### Functions
```python
detect_language(body)
```
```
English
```
___
```python
find_entities(body)
# still unprocessed result
```
```
b'{"documents":[{"id":"TextTest","entities":[{"name":"Conceived in Liberty","matches":[{"text":"conceived in Liberty","offset":91,"length":20}],"wikipediaLanguage":"en","wikipediaId":"Conceived in Liberty","wikipediaUrl":"https://en.wikipedia.org/wiki/Conceived_in_Liberty","bingId":"9540782b-ef0a-caed-5657-9706c6b8ce02"},{"name":"Earth","matches":[{"text":"earth","offset":1450,"length":5}],"wikipediaLanguage":"en","wikipediaId":"Earth","wikipediaUrl":"https://en.wikipedia.org/wiki/Earth","bingId":"6ddb3372-4801-5567-321e-e8a53bd774a4"},{"name":"God","matches":[{"text":"God","offset":1319,"length":3}],"wikipediaLanguage":"en","wikipediaId":"God","wikipediaUrl":"https://en.wikipedia.org/wiki/God","bingId":"d38371fb-8068-3139-6cb8-e9cfe3118da0"}]}],"errors":[]}'
```
____

```python
key_phrases(body)
```
```
  new nation
  people
  honored dead
  great civil war
  great battlefield
  great task
  living
  increased devotion
  measure of devotion
  brave men
  new birth of freedom
  government
  fathers
  final resting place
  proposition
  continent
  Liberty
  God
  world
  little note
  score
  years
  cause
  larger sense
  poor power
  unfinished work
  portion
  lives
  ground
  earth
```

```python
check_sentiment(body)
```
```
Document:TextTest = positive